# Importing necassary libraries

In [1]:
import numpy as np
import scipy.io
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import soundfile as sf
import random
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import math
import statistics




# Defining the function that reads the audios from a given path using soundfile and returns 4 lists : audios, freqs, filepaths and problematic files for potential debugging

In [2]:

def read_audios(directory):
    audios = []
    freqs = []
    filepaths = []
    problematic_files = []
    
    # Walking through the directory that contains the dataset and reading each file that has the .wav extension
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                filepath = os.path.join(root, file)
                filepaths.append(filepath)
                try:
                    # Read the audio file using soundfile
                    data, freq = sf.read(filepath)
                    audios.append(data)
                    freqs.append(freq)
                except Exception as e:
                    print(f"Encountered an error with file: {filepath}. Error: {e}")
                    problematic_files.append(filepath)
    
    return audios, freqs, filepaths, problematic_files

# Example usage
audios, freqs, filepaths, problematic_files = read_audios(r'German_wav')


# Defining the function that extracts the mfcc features then removes the frames of silence finally it saves the mffc features into a .txt file according to gender 

In [3]:
def extractMfccs_RemoveSilence_saveMfccs(audios,freqs,filepaths, directory):
    mfccs = []
 
    for audio, freq, filepath in zip(audios, freqs,filepaths):
        # extract the MFCC features
        mfcc_features = mfcc(audio, freq, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft= 2048, lowfreq=0,
                         highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=False)
        
        # calculate the energy
        energy = np.sum(mfcc_features**2, axis=1)
        # calculate the threshold for silence
        threshold = np.mean(energy) * 0.4
        #removing silence frames from mfccs
        voiced_indices = np.where(energy > threshold)[0]
        mfccs_voiced = mfcc_features[voiced_indices,:]
        mfccs.append(mfccs_voiced)
        
        # print the shape of the MFCCs before and after removing silence
        print(f"MFCCs before removing silence: {mfcc_features.shape}")
        print(f"MFCCs after removing silence: {mfccs_voiced.shape}")
        
       #saving mffcs 
       # extract the gender information from the file name
        gender = None
        if 'hommes' in filepath:
            gender = 'Hommes'
        elif 'femmes' in filepath:
            gender = 'Femmes'

        # save the MFCCs to the appropriate directory based on gender
        if gender is not None:
            gender_dir = os.path.join(directory, gender)
            if not os.path.exists(gender_dir):
                os.makedirs(gender_dir)
            mfcc_file = os.path.join(gender_dir,  os.path.splitext(os.path.basename(filepath))[0] + ".mfcc")
            np.savetxt(mfcc_file, mfccs_voiced, delimiter=',')
            
      
    
    return  mfccs

# Defining the function that splits the extraced mfccs into training and testing sets : 2/3 from male directory for training and 2/3 from female 

In [4]:
def train_test_split(mfcc_dir):
    # create separate lists for male and female file paths
    male_files = []
    female_files = []
    for root, dirs, files in os.walk(mfcc_dir):
        for file in files:
            if file.endswith('.mfcc'):
                if 'Hommes' in root:
                    male_files.append(os.path.join(root, file))
                elif 'Femmes' in root:
                    female_files.append(os.path.join(root, file))

    # shuffle the male and female lists independently
    random.shuffle(male_files)
    random.shuffle(female_files)

    # split the male and female lists into train and test based on the desired ratio
    male_train = male_files[:int(2/3*len(male_files))]
    male_test = male_files[int(2/3*len(male_files)):]

    female_train = female_files[:int(2/3*len(female_files))]
    female_test = female_files[int(2/3*len(female_files)):]

    
    
    # merge the train and test lists for both male and female
    train_files = male_train + female_train
    test_files = male_test + female_test

    # load the MFCC features from the saved files for the train and test sets
    train_mfccs = []
    test_mfccs = []

    for file in train_files:
        train_mfccs.append(np.loadtxt(file, delimiter=','))

    for file in test_files:
        test_mfccs.append(np.loadtxt(file, delimiter=','))

    # print the shapes of the train and test MFCC feature arrays
    print(f"Train male MFCCs shape: {np.array(male_train).shape}")
    print(f"Test male MFCCs shape: {np.array( male_test).shape}")
    print(f"Train female MFCCs shape: {np.array(female_train).shape}")
    print(f"Test female MFCCs shape: {np.array( female_test).shape}")
    print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
    print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")
    
    return train_mfccs, test_mfccs


# Defining the functions that train the different GMM models and than save them as a pkl file

In [5]:
def gmm16(train_mfccs):
    # Initialize the GMM model with 16 classes
    gmm = GaussianMixture(n_components=16, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model16_german.pkl')

    return gmm


In [6]:
def gmm32(train_mfccs):
    # Initialize the GMM model with 32 classes
    gmm = GaussianMixture(n_components=32, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model32_german.pkl')

    return gmm

In [7]:
def gmm64(train_mfccs):
    # Initialize the GMM model with 64 classes
    gmm = GaussianMixture(n_components=64, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model64_german.pkl')

    return gmm

In [8]:
def gmm128(train_mfccs):
    # Initialize the GMM model with 128 classes
    gmm = GaussianMixture(n_components=128, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model128_german.pkl')

    return gmm

In [9]:
def gmm256(train_mfccs):
    # Initialize the GMM model with 254 classes
    gmm = GaussianMixture(n_components=256, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model256_german.pkl')

    return gmm

# Getting the audios frequencies and filepaths from their directory using the function defined above

In [10]:
audios, freqs, filepaths, problematic_files = read_audios(r'German_wav')


# Extracting the mfcc features - Removing silence and saving the mfccs

In [11]:
mfccs = extractMfccs_RemoveSilence_saveMfccs(audios, freqs, filepaths,r'mfcc_german')

MFCCs before removing silence: (13265, 13)
MFCCs after removing silence: (13264, 13)
MFCCs before removing silence: (15872, 13)
MFCCs after removing silence: (15867, 13)
MFCCs before removing silence: (13618, 13)
MFCCs after removing silence: (13615, 13)
MFCCs before removing silence: (13649, 13)
MFCCs after removing silence: (13646, 13)
MFCCs before removing silence: (11981, 13)
MFCCs after removing silence: (11968, 13)
MFCCs before removing silence: (2752, 13)
MFCCs after removing silence: (2745, 13)
MFCCs before removing silence: (8827, 13)
MFCCs after removing silence: (8821, 13)
MFCCs before removing silence: (1177, 13)
MFCCs after removing silence: (1177, 13)
MFCCs before removing silence: (3192, 13)
MFCCs after removing silence: (3186, 13)
MFCCs before removing silence: (4095, 13)
MFCCs after removing silence: (4083, 13)
MFCCs before removing silence: (1551, 13)
MFCCs after removing silence: (1551, 13)
MFCCs before removing silence: (14976, 13)
MFCCs after removing silence: (149

MFCCs before removing silence: (625, 13)
MFCCs after removing silence: (621, 13)
MFCCs before removing silence: (784, 13)
MFCCs after removing silence: (781, 13)
MFCCs before removing silence: (1036, 13)
MFCCs after removing silence: (1031, 13)
MFCCs before removing silence: (1216, 13)
MFCCs after removing silence: (1213, 13)
MFCCs before removing silence: (1230, 13)
MFCCs after removing silence: (1224, 13)
MFCCs before removing silence: (1007, 13)
MFCCs after removing silence: (1003, 13)
MFCCs before removing silence: (791, 13)
MFCCs after removing silence: (790, 13)
MFCCs before removing silence: (1108, 13)
MFCCs after removing silence: (1105, 13)
MFCCs before removing silence: (1324, 13)
MFCCs after removing silence: (1321, 13)
MFCCs before removing silence: (885, 13)
MFCCs after removing silence: (880, 13)
MFCCs before removing silence: (877, 13)
MFCCs after removing silence: (874, 13)
MFCCs before removing silence: (697, 13)
MFCCs after removing silence: (694, 13)
MFCCs before rem

MFCCs before removing silence: (978, 13)
MFCCs after removing silence: (976, 13)
MFCCs before removing silence: (849, 13)
MFCCs after removing silence: (847, 13)
MFCCs before removing silence: (1007, 13)
MFCCs after removing silence: (1001, 13)
MFCCs before removing silence: (1374, 13)
MFCCs after removing silence: (1369, 13)
MFCCs before removing silence: (798, 13)
MFCCs after removing silence: (796, 13)
MFCCs before removing silence: (1173, 13)
MFCCs after removing silence: (1171, 13)
MFCCs before removing silence: (906, 13)
MFCCs after removing silence: (905, 13)
MFCCs before removing silence: (1511, 13)
MFCCs after removing silence: (1509, 13)
MFCCs before removing silence: (978, 13)
MFCCs after removing silence: (970, 13)
MFCCs before removing silence: (906, 13)
MFCCs after removing silence: (905, 13)
MFCCs before removing silence: (1281, 13)
MFCCs after removing silence: (1280, 13)
MFCCs before removing silence: (1050, 13)
MFCCs after removing silence: (1049, 13)
MFCCs before rem

## -----> We can see here that the size of the mfcc features has decreased after removing the frames ot silence

# Splitting into teest and train sets according to gender

In [12]:
train_mfccs, test_mfccs = train_test_split(r'mfcc_german')

Train male MFCCs shape: (138,)
Test male MFCCs shape: (70,)
Train female MFCCs shape: (10,)
Test female MFCCs shape: (5,)
Train MFCCs shape: (148,)
Test MFCCs shape: (75,)


C:\Users\HP\AppData\Local\Temp\ipykernel_13368\3397580047.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
C:\Users\HP\AppData\Local\Temp\ipykernel_13368\3397580047.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")


# Stacking vertically the train and test MFCC features so that we can fit the gmm models

In [13]:
#stack vertically the train MFCC features 
mfcc_train = []
for train_mfcc in train_mfccs:
    mfcc_train.append(train_mfcc)
mfcc_train = np.concatenate(mfcc_train, axis=0)

#stack vertically the test MFCC features 
mfcc_test = []
for test_mfcc in test_mfccs:
    mfcc_test.append(test_mfcc)
mfcc_test = np.concatenate(mfcc_test, axis=0)


# Saving the test set into a txt file 

In [14]:
#Save the test mfccs in a file
test_mfccs = np.vstack(test_mfccs)
test_mfccs = np.array(test_mfccs, dtype=float)
np.savetxt(r'germanTest', test_mfccs, delimiter=',')



In [15]:
mfcc_train.shape

(272103, 13)

# Training the different Gmm Models 

In [16]:
gmm16 = gmm16(mfcc_train)

In [17]:
gmm32 = gmm32(mfcc_train)

In [18]:
gmm64 = gmm64(mfcc_train)

In [19]:
gmm128= gmm128(mfcc_train)

In [20]:
gmm256= gmm256(mfcc_train)

# Evaluate the performance of each GMM model on the test set using the score_samples() function that returns an array containing the log-likelihood of each frame of the mfcc features

In [21]:
scores = []
for model in [gmm16, gmm32, gmm64, gmm128, gmm256]:
    score = model.score_samples(mfcc_test)
    scores.append(score)

# Print the scores
print('GMM16 score:', scores[0])
print('GMM32 score:', scores[1])
print('GMM64 score:', scores[2])
print('GMM128 score:', scores[3])
print('GMM256 score:', scores[4])



GMM16 score: [ 72.2914364   72.2914364  -54.87555354 ... -59.41894395 -58.15119954
 -56.79780782]
GMM32 score: [ 72.2914364   72.2914364  -52.29551112 ... -56.56640721 -52.37466656
 -51.28764862]
GMM64 score: [ 72.2914364   72.2914364  -48.27474488 ... -54.8706473  -49.8352651
 -49.36858118]
GMM128 score: [ 72.2914364   72.2914364  -53.87521104 ... -54.91679338 -47.62094746
 -46.97990054]
GMM256 score: [ 72.2914364   72.2914364  -45.49188467 ... -51.4573436  -41.56124324
 -41.79560501]


# Comparing the size of our mfcc_test set with the size of the scores array 

In [22]:
mfcc_test.shape

(135633, 13)

# Indeed the size of the scores array is the same 

In [23]:
 scores[0].shape

(135633,)

# In order to compare between the different GMM Models we need to calculate the score for the whole test set and we can do that by calculating the mean of the individual scores

In [24]:
#calculationg the score of the hole test set
print('GMM16 score:', scores[0].mean())
print('GMM32 score:', scores[1].mean())
print('GMM64 score:', scores[2].mean())
print('GMM128 score:', scores[3].mean())
print('GMM256 score:', scores[4].mean())


GMM16 score: -52.21698108758567
GMM32 score: -51.63131698380798
GMM64 score: -51.32815554953533
GMM128 score: -51.045380939824994
GMM256 score: -50.8501912396092


### From the results above we can see that the best score (the closest one to 0) is given by the model using 16 gaussians 